In [1]:
import os, sys, json, webbrowser, pprint
import spotipy 
import lyricsgenius
import pandas as pd
import nltk
import spotipy.util as util
import spotipy.oauth2 as oauth2
from json.decoder import JSONDecodeError
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from requests.exceptions import Timeout


C:\Users\Timmo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
# Define variables
username = 'g64129687775'
client_id = 'd576e9eb16044adbaa2d22688fc73dd0'
client_secret = '7b5cc4d0a7ce40ee9f8c0ea42aba241b'
redirect_uri = 'http://localhost/:8000'
scope='user-read-recently-played'

In [3]:
# Create tokens
token = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope,username=username,show_dialog=True)
genius_access_token = 'cQ15SgX_iCubNYTUW2pivTNrZbHmUIBHlzUHg2GEbDbrFUmzEVaRFE9BPvefe2io'

In [4]:
# Create API objects
spotifyObject = spotipy.Spotify(auth_manager=token)
geniusObject = lyricsgenius.Genius(genius_access_token)

In [5]:
# Create NLP analyser
analyser = SentimentIntensityAnalyzer()

In [6]:
# Get playlist
playlist = spotifyObject.playlist('spotify:playlist:37i9dQZF1DX2yvmlOdMYzV')
mood = 8
playlist_name = playlist['name']

In [7]:
results = playlist['tracks']
p = results['items']
while results['next']:
    results = spotifyObject.next(results)
    p.extend(results['items'])

In [8]:
if mood == 1:
    mood = 'Happy'
elif mood == 2:
    mood = 'Sad'
elif mood == 3:
    mood = 'Calm'
elif mood == 4:
    mood = 'Sleepy'
elif mood == 5:
    mood = 'Energised'
elif mood == 6:
    mood = 'Aroused'
elif mood == 7:
    mood = 'Angry'
elif mood == 8:
    mood = 'Chill'

In [9]:

# p_item = playlist['tracks']['items']

# track_id = p_item[0]['track']['id']
# track_name = p_item[0]['track']['name']
# track_artists = p_item[0]['track']['artists']
# song = geniusObject.search_song(title=track_name, artist=track_artists[0]['name'])
# annotation = geniusObject.song_annotations(song.id)
# print(song.lyrics+'\n')
# print(annotation)



In [10]:
geniusObject.timeout = 15
songs = []

In [11]:
def retrieveLyrics(track_name, track_artists):
    annotation = None
    lyric_score = None
    annotation_score = None

    retries = 0
    while retries < 3:            
        try:
            # Think of a possible way to search with all featuring artists to increase chance of finding song
            song = geniusObject.search_song(title=track_name, artist=track_artists[0]['name'])
            break
        except Timeout as e:
            retries += 1
            continue

    if retries==3:
        print('Failed to fetch song lyrics for: ', track_name)
        return None

    # Not using NLP
    nlp_lyrics = lyric_score['compound'] if lyric_score else None,
    nlp_annotations = annotation_score['compound'] if annotation_score else None,
    
    return(nlp_lyrics, nlp_annotations)



In [12]:
def retrieveSongs(x):
    # Get song id, name and artist(s)
    track_id = x['track']['id']
    track_name = x['track']['name']
    track_artists = x['track']['artists']

    # Get features
    features = spotifyObject.audio_features(track_id)[0]

    # Create song dictionary
    songDict = {'name' : track_name, 'artists' : [d['name'] for d in track_artists]}

    # Add song features to dictionary
    for feat in features.keys():
        songDict[feat] = features[feat]

    # Return lyrics
    lyrics = retrieveLyrics(track_name, track_artists)
    if lyrics:
        songDict['nlp_lyrics'] = lyrics[0]
        songDict['nlp_annotations'] = lyrics[1]

    songDict['mood'] = mood

    return songDict


In [13]:
songs = [retrieveSongs(x) for x in p]

Searching for "IN THE AIR" by DESTIN CONRAD...
Done.
Searching for "Requiem" by Khamari...
Done.
Searching for "Good & Plenty - Remix" by Lucky Daye...
Done.
Searching for "69 Camaro" by Kyle Dion...
Done.
Searching for "dirty dancer" by Orion Sun...
Done.
Searching for "say im ur luv" by UMI...
Done.
Searching for "Exit" by Sebastian Mikael...
Done.
Searching for "Golden pt. 2 (feat. Mereba)" by Berhana...
Done.
Searching for "Silence" by Marco McKinnis...
Done.
Searching for "Fingers Crossed" by Léonie Barbot...
Done.
Searching for "Skin Tight (feat. Steve Lacy)" by Ravyn Lenae...
Done.
Searching for "We Might Even Be Falling In Love (Duet) - Spotify Singles" by Victoria Monét...
Done.
Searching for "IDWL" by CUBE...
Done.
Searching for "All For You" by Amaria...
Done.
Searching for "Addiction" by Emanuel...
Done.
Searching for "Jealous" by Khamari...
Done.
Searching for "PATTERNS" by Chelan...
Done.
Searching for "D4U - Spotify Singles" by Avenoir...
Done.
Searching for "Verbs (feat

In [14]:
df = pd.DataFrame(songs)
df

,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,nlp_lyrics,nlp_annotations
0,IN THE AIR,[DESTIN CONRAD],0.794,0.389,8,-6.132,1,0.0545,0.0375,0.001940,...,114.920,audio_features,2APwouMoPiiija7dD41I9f,spotify:track:2APwouMoPiiija7dD41I9f,https://api.spotify.com/v1/tracks/2APwouMoPiii...,https://api.spotify.com/v1/audio-analysis/2APw...,136696,4,"(None,)","(None,)"
1,Requiem,[Khamari],0.354,0.380,5,-9.520,0,0.0413,0.2730,0.000762,...,77.605,audio_features,2JydmjbQ7nGdVI6HbumkLA,spotify:track:2JydmjbQ7nGdVI6HbumkLA,https://api.spotify.com/v1/tracks/2JydmjbQ7nGd...,https://api.spotify.com/v1/audio-analysis/2Jyd...,284267,4,"(None,)","(None,)"
2,Good & Plenty - Remix,"[Lucky Daye, Masego, Alex Isley, Jack Dine]",0.629,0.579,9,-6.543,0,0.1070,0.8110,0.003660,...,135.860,audio_features,6T3Ebo7EOh8cUOyE4OhFpp,spotify:track:6T3Ebo7EOh8cUOyE4OhFpp,https://api.spotify.com/v1/tracks/6T3Ebo7EOh8c...,https://api.spotify.com/v1/audio-analysis/6T3E...,204706,4,"(None,)","(None,)"
3,69 Camaro,[Kyle Dion],0.610,0.416,5,-7.357,1,0.0612,0.6720,0.000000,...,82.950,audio_features,70XkGdEfN4RM7bEwaNaBao,spotify:track:70XkGdEfN4RM7bEwaNaBao,https://api.spotify.com/v1/tracks/70XkGdEfN4RM...,https://api.spotify.com/v1/audio-analysis/70Xk...,154818,4,"(None,)","(None,)"
4,dirty dancer,[Orion Sun],0.702,0.525,7,-5.028,0,0.0313,0.3760,0.066700,...,113.856,audio_features,1ciQU7ZQGHq129m3njp9en,spotify:track:1ciQU7ZQGHq129m3njp9en,https://api.spotify.com/v1/tracks/1ciQU7ZQGHq1...,https://api.spotify.com/v1/audio-analysis/1ciQ...,169720,4,"(None,)","(None,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Best Feature,"[CUBE, Devin Morrison]",0.527,0.618,11,-6.659,0,0.2670,0.1770,0.000005,...,85.557,audio_features,0ZrdBusEajQaIKl7jdJ8u4,spotify:track:0ZrdBusEajQaIKl7jdJ8u4,https://api.spotify.com/v1/tracks/0ZrdBusEajQa...,https://api.spotify.com/v1/audio-analysis/0Zrd...,203721,4,"(None,)","(None,)"
96,Gone Away,[Tia Gordon],0.689,0.354,9,-9.429,0,0.0322,0.5050,0.000102,...,89.975,audio_features,20TQFjdslMaSvYRJkBYxNA,spotify:track:20TQFjdslMaSvYRJkBYxNA,https://api.spotify.com/v1/tracks/20TQFjdslMaS...,https://api.spotify.com/v1/audio-analysis/20TQ...,290427,4,"(None,)","(None,)"
97,Taj Mahal,[Kent Jamz],0.581,0.533,7,-6.036,1,0.0725,0.0276,0.000005,...,146.546,audio_features,5VAxNirVOEJXjY6cWhAcOy,spotify:track:5VAxNirVOEJXjY6cWhAcOy,https://api.spotify.com/v1/tracks/5VAxNirVOEJX...,https://api.spotify.com/v1/audio-analysis/5VAx...,216333,4,"(None,)","(None,)"
98,THINK TWICE,[Jordan Ward],0.568,0.439,4,-10.846,1,0.0903,0.8860,0.026600,...,88.196,audio_features,0SjibUhSN9ozHhwAVpHMAg,spotify:track:0SjibUhSN9ozHhwAVpHMAg,https://api.spotify.com/v1/tracks/0SjibUhSN9oz...,https://api.spotify.com/v1/audio-analysis/0Sji...,131173,4,"(None,)","(None,)"


: 

In [14]:
songs = map(retrieveSongs, p)
songs = list(songs)
print(songs)

Searching for "IN THE AIR" by DESTIN CONRAD...
Done.
Searching for "Requiem" by Khamari...
Done.
Searching for "Good & Plenty - Remix" by Lucky Daye...
Done.
Searching for "69 Camaro" by Kyle Dion...
Done.
Searching for "dirty dancer" by Orion Sun...
Done.
Searching for "say im ur luv" by UMI...
Done.
Searching for "Exit" by Sebastian Mikael...
Done.
Searching for "Golden pt. 2 (feat. Mereba)" by Berhana...
Done.
Searching for "Silence" by Marco McKinnis...
Done.
Searching for "Fingers Crossed" by Léonie Barbot...
Done.
Searching for "Skin Tight (feat. Steve Lacy)" by Ravyn Lenae...
Done.
Searching for "We Might Even Be Falling In Love (Duet) - Spotify Singles" by Victoria Monét...
Done.
Searching for "IDWL" by CUBE...
Done.
Searching for "All For You" by Amaria...
Done.
Searching for "Addiction" by Emanuel...
Done.
Searching for "Jealous" by Khamari...
Done.
Searching for "PATTERNS" by Chelan...
Done.
Searching for "D4U - Spotify Singles" by Avenoir...
Done.
Searching for "Verbs (feat

In [ ]:
for x in p:
    track_id = x['track']['id']
    track_name = x['track']['name']
    track_artists = x['track']['artists']
    
    
    features = spotifyObject.audio_features(track_id)[0]


    try:
        songs.append({
        # Spotify features
        'name' : track_name,
        'artists' : [d['name'] for d in track_artists],
        'acousticness' : features['acousticness'],
        'danceability' : features['danceability'],
        'duration_ms' : features['duration_ms'],
        'energy' : features['energy'],
        'instrumentalness' : features['instrumentalness'],
        'key' : features['key'],
        'liveness' : features['liveness'],
        'loudness' : features['loudness'],
        'mode' : features['mode'],
        'speechiness' : features['speechiness'],
        'tempo' : features['tempo'],
        'time_signature' : features['time_signature'],
        'valence' : features['valence'],
        

        # Genius features
        # 'lyrics' : song.lyrics if song else None,
        # 'annotations' : annotation if annotation else None,
        'nlp_lyrics' : lyric_score['compound'] if lyric_score else None,
        'nlp_annotations' : annotation_score['compound'] if annotation_score else None,
        

        # Rework this
        'valence+nlp' : round(features['valence']+(lyric_score['compound']+annotation_score['compound'])/100, 4) if song else features['valence'],
        'mood': mood 
        })
    except Exception as e:
        print('############################')
        print(e)
        print('Error 2 For', track_name)
        print('############################')


############################
name 'lyric_score' is not defined
Error 2 For IN THE AIR
############################
############################
name 'lyric_score' is not defined
Error 2 For Requiem
############################
############################
name 'lyric_score' is not defined
Error 2 For Good & Plenty - Remix
############################
############################
name 'lyric_score' is not defined
Error 2 For 69 Camaro
############################
############################
name 'lyric_score' is not defined
Error 2 For dirty dancer
############################
############################
name 'lyric_score' is not defined
Error 2 For say im ur luv
############################
############################
name 'lyric_score' is not defined
Error 2 For Golden pt. 2 (feat. Mereba)
############################
############################
name 'lyric_score' is not defined
Error 2 For Exit
############################
############################
name 'lyric_score' is not de